# AAFG-SBIR — Colab/Jupyter Notebook

Ready-to-run steps for training/evaluating the Attention-Augmented FG-SBIR implementation.

**Options to get the code:**
- **GitHub** (recommended):
  1) Create a new empty repo on your account and push the code from this ZIP.
  2) Replace `<your-username>`/`<your-repo>` below and run the `git clone` cell.
- **Local upload**: If running locally, open this notebook from inside the project root.

In [ ]:
#@title Clone your GitHub repo here (after you push)
GIT_REPO = "https://github.com/USERNAME/REPO.git"  #@param {type:"string"}
GIT_DIR  = ""  #@param {type:"string"}

import os, sys, subprocess

if GIT_REPO and GIT_REPO != "https://github.com/USERNAME/REPO.git":
    print("Cloning:", GIT_REPO)
    !git clone $GIT_REPO
    if not GIT_DIR:
        # infer directory from repo URL
        GIT_DIR = os.path.splitext(os.path.basename(GIT_REPO))[0]
    %cd $GIT_DIR
else:
    print("[Info] Set GIT_REPO to your GitHub repository URL after you push the code.")


In [ ]:

#@title Install requirements
!python -m pip install -q -r requirements.txt


In [ ]:

#@title Check GPU / environment
import torch, platform, sys
print("Python:", sys.version)
print("Platform:", platform.platform())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))


## Prepare tiny smoke dataset
This creates minimal images and CSVs to verify the training/evaluation loop without downloading full datasets.

In [ ]:

#@title Create tiny sample images and CSVs (Sketchy-style layout)
from PIL import Image, ImageDraw
import os, pandas as pd

root = "DATA/Sketchy"
sk_dir = os.path.join(root, "sketches")
im_dir = os.path.join(root, "images")
os.makedirs(sk_dir, exist_ok=True)
os.makedirs(im_dir, exist_ok=True)

# Make a couple of tiny 224x224 images
def make_img(path, text):
    img = Image.new("RGB", (224,224), color=(240,240,240))
    d = ImageDraw.Draw(img)
    d.text((10,10), text, fill=(0,0,0))
    img.save(path)

make_img(os.path.join(sk_dir, "example1.png"), "sketch-1")
make_img(os.path.join(im_dir, "example1.jpg"), "image-1")
make_img(os.path.join(sk_dir, "example2.png"), "sketch-2")
make_img(os.path.join(im_dir, "example2.jpg"), "image-2")

# Build tiny CSVs
import pandas as pd
train = pd.DataFrame([
    {"path_sketch":"sketches/example1.png", "path_image":"images/example1.jpg", "label":0},
])
val = pd.DataFrame([
    {"path_sketch":"sketches/example2.png", "path_image":"images/example2.jpg", "label":0},
])
test = val.copy()

train.to_csv(os.path.join(root, "tiny_train.csv"), index=False)
val.to_csv(os.path.join(root, "tiny_val.csv"), index=False)
test.to_csv(os.path.join(root, "index_test.csv"), index=False)

print("Created tiny dataset under", root)


## Run unit test (forward pass)
Verifies shapes and L2-normalization.

In [ ]:

!python tests/test_forward.py


## Train (smoke run)
Two epochs on the tiny dataset to validate the training loop and logging.

In [ ]:

!python scripts/train.py \
  --data_root DATA/Sketchy \
  --train_csv tiny_train.csv \
  --val_csv tiny_val.csv \
  --epochs 2 \
  --batch_size 2 \
  --lr 1e-4 \
  --margin 0.2 \
  --log_dir runs/smoke


## Evaluate on the tiny test set
This will print mAP and P@K for the small sample.

In [ ]:

!python scripts/eval.py \
  --data_root DATA/Sketchy \
  --test_csv index_test.csv \
  --ckpt checkpoints/best.pt \
  --batch_size 2
